In [1]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [2]:
# 데이터 경로 지정
import sys
sys.path.append('gdrive/My Drive/RogerHeederer/NLP_KR/6.CHATBOT_transformer')

In [3]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.8MB/s 
     |████████████████████████████████| 460kB 46.9MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [5]:
from PreprocessForTransformer import *

In [6]:
BASE_PATH = 'gdrive/My Drive/RogerHeederer/NLP_KR/6.CHATBOT_transformer'
DATA_PATH = BASE_PATH + '/data_in/ChatBotData.csv'
VOCAB_PATH = BASE_PATH + '/data_in/vocabulary.txt'

In [8]:
inputs, outputs = load_data(DATA_PATH)

In [10]:
inputs[0:5], outputs[0:5]

(['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네'],
 ['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.'])

In [11]:
#단어 사전 만들기
char2idx, idx2char, vocab_size = load_vocabulary(DATA_PATH, VOCAB_PATH, tokenize_as_morph=True)

100%|██████████| 11823/11823 [00:45<00:00, 257.88it/s]


#1.인덱스 데이터 구성하기#
만들어진 단어 사전을 가지고, 모델에 학습시킬 인덱스 데이터를 구성한다

In [14]:
#트랜스포머는의 학습데이터는 형태소 단위로 토크나이징 된 방식으로 전처리 진행한다
index_inputs, input_seq_len = enc_processing(inputs, char2idx, tokenize_as_morph=True)
index_outputs, output_seq_len = dec_output_processing(outputs, char2idx, tokenize_as_morph=True)
index_targets = dec_target_processing(outputs, char2idx, tokenize_as_morph=True)

100%|██████████| 11823/11823 [00:45<00:00, 261.50it/s]


#2.단어 사전. 특별토큰을 딕셔너리 객체에 담기

In [16]:
# 모델 학습 및 추론에 활용하기 위한 단어 사전과 특별 토큰들을 딕셔너리 객체에 담아둔다
data_configs = {}
data_configs['char2idx'] = char2idx
data_configs['idx2char'] = idx2char
data_configs['vocab_size'] = vocab_size
data_configs['pad_symbol'] = PAD
data_configs['std_symbol'] = STD
data_configs['end_symbol'] = END
data_configs['unk_symbol'] = UNK

#3. 인덱스 데이터를 .npy 파일로 저장/ config 파일을 json형태로 저장#

In [17]:
DATA_IN_PATH = BASE_PATH + '/data_in/'
TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'
DATA_CONFIGS = 'data_configs.json'

np.save(open(DATA_IN_PATH + TRAIN_INPUTS, 'wb'), index_inputs)
np.save(open(DATA_IN_PATH + TRAIN_OUTPUTS , 'wb'), index_outputs)
np.save(open(DATA_IN_PATH + TRAIN_TARGETS , 'wb'), index_targets)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'))

In [24]:
char2idx['차별'], idx2char[2658]

(2658, '차별')

#모델 학습용 데이터 준비 끝#